In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
faces = pd.read_csv('FGNet-LOPO.csv')


In [97]:
faces['ageclass'] = faces.age.apply(
    lambda r: 0 if r < 18 else 1
).astype(int)
faces.head()

,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,...,b104,b105,b106,b107,b108,b109,age,ID,Gender_0M_1F,ageclass
0,0.585770,-0.075982,-0.276594,0.076490,0.189404,-0.152451,0.003264,-0.366677,0.148862,-0.078316,...,0.012785,0.014379,-0.032976,-0.005910,0.015997,-0.002833,5,1,0,0
1,0.117512,-0.344377,0.413923,-0.088210,0.152117,-0.020675,-0.085257,-0.122578,0.085524,-0.035403,...,0.006724,0.014217,0.001321,-0.005616,-0.018197,-0.000073,8,1,0,0
2,0.248636,-0.209122,0.516829,-0.124100,-0.002087,0.165375,-0.033023,0.035212,0.156937,-0.028065,...,0.023377,0.028616,-0.009582,-0.020722,0.003676,-0.015716,10,1,0,0
3,0.136742,-0.095098,0.428192,-0.125278,-0.117103,0.080187,0.109120,-0.068001,0.185257,-0.001298,...,0.021750,0.006348,-0.006021,-0.005336,0.002317,-0.014843,14,1,0,0
4,-0.371003,-0.164112,0.431468,-0.163736,-0.217533,0.083152,-0.053181,-0.169808,0.178207,-0.000211,...,0.008738,0.002282,-0.021130,-0.009988,-0.016178,-0.003646,16,1,0,0


In [105]:

x = faces.drop(["age", "ID", "Gender_0M_1F"], axis = 1)
y = faces[["age", "ID", "Gender_0M_1F", "ageclass"]]
y_young = y[y.ageclass == 0]
y_old = y[y.ageclass == 1]
x_young = x[x.ageclass == 0].drop(["ageclass"], axis = 1)
x_old = x[x.ageclass == 1].drop(["ageclass"], axis = 1)
x = x.drop(["ageclass"], axis = 1)
x_young_train, x_young_test, y_young_train, y_young_test = train_test_split(x_young, y_young, test_size = .2)
x_old_train, x_old_test, y_old_train, y_old_test = train_test_split(x_old, y_old, test_size = .2)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2)


In [107]:
logReg = LinearDiscriminantAnalysis().fit(x_train, y_train["ageclass"])
reg = LinearRegression().fit(x_young_train, y_young_train["age"])
regTwo = LinearRegression().fit(x_old_train, y_old_train["age"])

score = 0
total = 0
error = 0

for index, row in x_test.iterrows():
    row = row.values.reshape(1,-1)
    if reg.predict(row) == 0:
        age = np.round(reg.predict(row))
        if age == y_test["age"][index]:
            score += 1
        else:
            error += abs(y_test["age"][index] - age)
        total += 1
    else:
        age = np.round(regTwo.predict(row))
        if age == y_test["age"][index]:
            score += 1
        else:
            error += abs(y_test["age"][index] - age)
        total += 1
        
print(score/total)
print(error/total)

0.04477611940298507
[9.92537313]


     age  ID  Gender_0M_1F  ageclass
972    7  80             0         0
714    7  60             1         0
331   36  27             1         1
967    2  80             0         0
501    6  41             0         0
..   ...  ..           ...       ...
500    4  41             0         0
983    4  81             0         0
76    67   6             0         1
899   10  75             0         0
805   10  68             0         0

[801 rows x 4 columns]
